<a href="https://colab.research.google.com/github/HoangHungLN/MachineLearning_Assignment/blob/main/Extended_Assignment/notebooks/Extended_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
import json
import os

!wget https://raw.githubusercontent.com/HoangHungLN/MachineLearning_Assignment/refs/heads/main/Extended_Assignment/data/train.json
